# Test Authoring 
- analysis requiremetn
- generate test script

In [39]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [40]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings=HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

In [41]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [57]:
loader=DirectoryLoader("./data",glob="./*.txt",loader_cls=TextLoader)
docs=loader.load()
text_splitter=RecursiveCharacterTextSplitter(
chunk_size=500,
chunk_overlap=50
)

new_docs=text_splitter.split_documents(documents=docs)

In [58]:
new_docs

[Document(metadata={'source': 'data\\API.txt'}, page_content='package com.automation.library;\nimport io.restassured.RestAssured;\nimport io.restassured.response.Response;\nimport io.restassured.specification.RequestSpecification;\nimport java.util.Map;\n\npublic class ApiClient {\n\n    private String baseUrl;\n    private RequestSpecification request;'),
 Document(metadata={'source': 'data\\API.txt'}, page_content='/**\n     * Constructor to initialize base URL\n     * @param baseUrl API base URL\n     */\n    public ApiClient(String baseUrl) {\n        this.baseUrl = baseUrl;\n        this.request = RestAssured.given().baseUri(baseUrl);\n    }\n\n    /**\n     * Set common headers for API calls\n     * @param headers Map of headers\n     * @return ApiClient instance\n     */\n    public ApiClient setHeaders(Map<String, String> headers) {\n        request.headers(headers);\n        return this;\n    }'),
 Document(metadata={'source': 'data\\API.txt'}, page_content='/**\n     * Set re

In [59]:
doc_string=[doc.page_content for doc in new_docs]
db=Chroma.from_documents(new_docs,embeddings)

In [62]:
retriever=db.as_retriever(search_kwargs={"k": 3})

In [64]:
retriever.invoke("create API Test for endpoint https://api.restful-api.dev/objects, its GET call. validate status code as 200")

[Document(metadata={'source': 'data\\API.txt'}, page_content='package com.automation.library;\nimport io.restassured.RestAssured;\nimport io.restassured.response.Response;\nimport io.restassured.specification.RequestSpecification;\nimport java.util.Map;\n\npublic class ApiClient {\n\n    private String baseUrl;\n    private RequestSpecification request;'),
 Document(metadata={'source': 'data\\API.txt'}, page_content='Object actualValue = response.jsonPath().get(key);\nreturn expectedValue.equals(actualValue);\n}\n}'),
 Document(metadata={'source': 'data\\API.txt'}, page_content='Object actualValue = response.jsonPath().get(key);\nreturn expectedValue.equals(actualValue);\n}\n}')]

[Document(metadata={'source': 'data\\API.txt'}, page_content='package com.automation.library;\nimport io.restassured.RestAssured;\nimport io.restassured.response.Response;\nimport io.restassured.specification.RequestSpecification;\nimport java.util.Map;\n\npublic class ApiClient {\n\n    private String baseUrl;\n    private RequestSpecification request;'),
 Document(metadata={'source': 'data\\API.txt'}, page_content='Object actualValue = response.jsonPath().get(key);\nreturn expectedValue.equals(actualValue);\n}\n}'),
 Document(metadata={'source': 'data\\API.txt'}, page_content='Object actualValue = response.jsonPath().get(key);\nreturn expectedValue.equals(actualValue);\n}\n}')]

In [55]:
from langchain import hub
import pprint

prompt = hub.pull("rlm/rag-prompt")
prompt.messages
pprint.pprint(prompt.messages)


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [69]:
from langchain_core.prompts import ChatPromptTemplate

custom_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert Test Automation Engineer. You role is to create API Test scripts in Java language based on the reference methods mentioned in  ApiClient class  reusable methods. "
    "DO not create pom.xml and associated dependencies and the base APIClient library"
    "Do not create any verbose/text as output"
    "Create modular script as per the details given by user "),
    ("user", "Context:\n{context}\n\nQuestion:\n{question}")
])

In [70]:
# Chain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_prompt
    | model
    | StrOutputParser()
)

In [71]:
result=rag_chain.invoke("Create API Test, for endpoint https://api.restful-api.dev/objects,  Operation is GET call. Validate status code as 200")

In [72]:
pprint.pprint(result)

('```java\n'
 'package com.automation.tests;\n'
 '\n'
 'import com.automation.library.ApiClient;\n'
 'import io.restassured.response.Response;\n'
 'import org.junit.jupiter.api.BeforeAll;\n'
 'import org.junit.jupiter.api.DisplayName;\n'
 'import org.junit.jupiter.api.Test;\n'
 'import static org.junit.jupiter.api.Assertions.assertTrue;\n'
 '\n'
 'public class ObjectApiTest {\n'
 '\n'
 '    private static ApiClient apiClient;\n'
 '    private static final String BASE_URL = "https://api.restful-api.dev";\n'
 '    private static final String OBJECTS_ENDPOINT = "/objects";\n'
 '\n'
 '    @BeforeAll\n'
 '    static void setup() {\n'
 '        apiClient = new ApiClient(BASE_URL);\n'
 '    }\n'
 '\n'
 '    @Test\n'
 '    @DisplayName("Verify GET /objects returns status code 200")\n'
 '    void testGetObjectsStatusCode() {\n'
 '        Response response = apiClient.get(OBJECTS_ENDPOINT);\n'
 '        boolean isStatusCodeValid = apiClient.validateStatusCode(response, '
 '200);\n'
 '        ass